In [23]:
import yfinance as yf
import pandas as pd
import numpy as np
# from qiskit.primitives import sampler    -> Will be Depreciated

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Sampler  
from qiskit_aer import Aer
from qiskit_aer import backends


In [24]:
#yf = yahoo finance
import yfinance as yf

#(NSE India stocks with .NS suffix)
stocks = ["TCS.NS", "INFY.NS", "HDFCBANK.NS", "RELIANCE.NS", "ITC.NS", "TATAMOTORS.NS", "BAJFINANCE.NS"]

# Download historical stock data for 3 years
df = yf.download(stocks, start="2021-01-01", end="2024-01-01")

#WE ARE USING ONLY THE CLOSE PRICE OF THE STOCKS
df = df['Close']
print(df.head())

[*********************100%***********************]  7 of 7 completed

Ticker      BAJFINANCE.NS  HDFCBANK.NS      INFY.NS      ITC.NS  RELIANCE.NS  \
Date                                                                           
2021-01-01    5204.157227  1368.017578  1148.968262  179.427795   898.454590   
2021-01-04    5141.127441  1359.329712  1174.309448  179.050232   899.968994   
2021-01-05    5045.326172  1369.601440  1179.368530  177.414124   888.780701   
2021-01-06    4957.902832  1363.697632  1168.703613  172.337936   865.341736   
2021-01-07    5007.873047  1359.569580  1150.518066  170.156448   863.940369   

Ticker      TATAMOTORS.NS       TCS.NS  
Date                                    
2021-01-01     185.348572  2713.368408  
2021-01-04     190.118927  2816.407959  
2021-01-05     192.007202  2866.028809  
2021-01-06     194.193604  2827.573730  
2021-01-07     195.535294  2810.245850  


In [25]:
"""we are calculating the daily return and the covariance matrix , the matrix is imp cause it tells us if our portfolio is well balanced or not
if the covariance is high then the stocks are not well balanced and if the covariance is low then the stocks are well balanced"""

returns = df.pct_change()
mean_returns = returns.mean()
cov_mat = returns.cov()


In [26]:
#building the qubo matrix
assets = len(stocks)
Q = np.zeros([assets, assets]) #init the qubo matrix
penalty  = np.mean(abs(mean_returns)) / np.mean(abs(cov_mat.values))
for i in range(assets):
    Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]
    for j in range(i+1, assets):
        Q[i,j] = -cov_mat.iloc[i,j]
        Q[j,i] = -cov_mat.iloc[i,j]

print(Q)   


[[ 2.12947677e-03 -1.19680321e-04 -8.72512558e-05 -7.81370277e-05
  -1.11089948e-04 -1.68183298e-04 -7.11395395e-05]
 [-1.19680321e-04  1.14790728e-03 -6.88003117e-05 -4.48002164e-05
  -7.81412631e-05 -1.13221232e-04 -5.53348666e-05]
 [-8.72512558e-05 -6.88003117e-05  1.28335483e-03 -4.03061746e-05
  -7.01201670e-05 -1.03995038e-04 -1.38913391e-04]
 [-7.81370277e-05 -4.48002164e-05 -4.03061746e-05  1.01683189e-04
  -6.32902560e-05 -1.06738913e-04 -2.94256613e-05]
 [-1.11089948e-04 -7.81412631e-05 -7.01201670e-05 -6.32902560e-05
   1.09924996e-03 -1.34463086e-04 -5.77973778e-05]
 [-1.68183298e-04 -1.13221232e-04 -1.03995038e-04 -1.06738913e-04
  -1.34463086e-04  2.29030269e-03 -9.63109095e-05]
 [-7.11395395e-05 -5.53348666e-05 -1.38913391e-04 -2.94256613e-05
  -5.77973778e-05 -9.63109095e-05  8.11314521e-04]]


C:\Users\kumar\AppData\Local\Temp\ipykernel_14980\1286306283.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  Q[i,i] = -mean_returns[i]+penalty*cov_mat.iloc[i,i]


In [27]:
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.primitives import BackendSampler

qubo = QuadraticProgram()
for i in range(assets):
    qubo.binary_var(f"x{i}")

linear_terms = {f"x{i}": Q[i, i] for i in range(assets)}
quadratic_terms = {(f"x{i}", f"x{j}"): Q[i, j] for i in range(assets) for j in range(i + 1, assets)}
qubo.minimize(linear=linear_terms, quadratic=quadratic_terms)

# backend = Aer.get_backend('qasm_simulator')
sampler = Sampler() 
qaoa = QAOA(sampler=sampler, optimizer=COBYLA(), reps=1)

# Convert QUBO to Ising model (handled internally by MinimumEigenOptimizer)
meo = MinimumEigenOptimizer(qaoa)
result = meo.solve(qubo)  

print(result)

fval=0.0, x0=0.0, x1=0.0, x2=0.0, x3=0.0, x4=0.0, x5=0.0, x6=0.0, status=SUCCESS


In [28]:
# import qiskit_optimization.applications
# help(qiskit_optimization.applications)